In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from erisk.utils import get_spark

bucket = "gs://dsgt-clef-erisk-2024"
spark = get_spark(cores=2, memory="4g")
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/24 17:01:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/24 17:01:36 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
rels_consensus = spark.read.csv(
    f"{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv",
    header=True,
    inferSchema=True,
)
rels_consensus.show(5)

+-----+---+-------------+---+
|query| q0|        docid|rel|
+-----+---+-------------+---+
|    1|  0|s_405_1279_15|  1|
|    1|  0| s_2519_356_0|  0|
|    1|  0|  s_2038_51_7|  1|
|    1|  0|   s_975_61_2|  0|
|    1|  0|  s_577_923_1|  1|
+-----+---+-------------+---+
only showing top 5 rows



In [9]:
rels_consensus.groupBy("q0").count().show()

+---+-----+
| q0|count|
+---+-----+
|  0|21580|
+---+-----+



In [8]:
rels_consensus.groupBy("query", "rel").count().orderBy("query").show(n=50)

+-----+---+-----+
|query|rel|count|
+-----+---+-----+
|    1|  0|  931|
|    1|  1|  179|
|    2|  1|  105|
|    2|  0| 1045|
|    3|  1|  161|
|    3|  0|  812|
|    4|  0|  916|
|    4|  1|   97|
|    5|  0|  746|
|    5|  1|   83|
|    6|  1|   21|
|    6|  0| 1058|
|    7|  1|  158|
|    7|  0|  847|
|    8|  1|   76|
|    8|  0|  996|
|    9|  1|  260|
|    9|  0|  693|
|   10|  1|  231|
|   10|  0|  752|
|   11|  1|   69|
|   11|  0| 1011|
|   12|  1|   71|
|   12|  0| 1006|
|   13|  0| 1049|
|   13|  1|   61|
|   14|  0|  996|
|   14|  1|   71|
|   15|  1|  130|
|   15|  0|  952|
|   16|  0|  735|
|   16|  1|  203|
|   17|  1|   94|
|   17|  0|  953|
|   18|  1|  103|
|   18|  0|  881|
|   19|  1|   83|
|   19|  0|  941|
|   20|  0|  910|
|   20|  1|  123|
|   21|  0|  874|
|   21|  1|   97|
+-----+---+-----+



In [18]:
from pyspark.sql import functions as F

(
    rels_consensus.groupBy("docid")
    .count()
    .withColumnRenamed("count", "rel_count")
    .groupBy("rel_count")
    .count()
    .orderBy(F.desc("count"))
).show()

+---------+-----+
|rel_count|count|
+---------+-----+
|        1|13535|
|        2| 1636|
|        3|  471|
|        4|  199|
|        5|   90|
|        7|   45|
|        6|   43|
|        8|   29|
|        9|   25|
|       10|   16|
|       20|    9|
|       11|    8|
|       14|    8|
|       19|    5|
|       12|    5|
|       13|    5|
|       18|    5|
|       17|    4|
|       15|    4|
|       16|    4|
+---------+-----+
only showing top 20 rows



In [19]:
# build the tfidf model
tfidf_df = spark.read.parquet(f"{bucket}/task1/parquet/combined_tfidf")
tfidf_df.printSchema()

root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hashingtf: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- tfidf: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [23]:
# join the tfidf with the relevance judgements
tfidf_df.select("DOCNO", "filename").where("dataset = 'train'").show(n=5)

+-------+--------+
|  DOCNO|filename|
+-------+--------+
|s_0_0_0|s_0.trec|
|s_0_1_0|s_0.trec|
|s_0_2_0|s_0.trec|
|s_0_3_0|s_0.trec|
|s_0_3_1|s_0.trec|
+-------+--------+
only showing top 5 rows



In [29]:
from pyspark.ml.functions import array_to_vector

train_df = tfidf_df.join(
    rels_consensus,
    tfidf_df.DOCNO == rels_consensus.docid,
    how="inner",
)
train_df.printSchema()
train_df.show(5)
train_df.count()

root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hashingtf: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- tfidf: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- query: integer (nullable = true)
 |-- q0: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)



+----------+--------------------+---------+-------+--------------------+--------------------+--------------------+-----+---+----------+---+
|     DOCNO|                TEXT| filename|dataset|               words|           hashingtf|               tfidf|query| q0|     docid|rel|
+----------+--------------------+---------+-------+--------------------+--------------------+--------------------+-----+---+----------+---+
|  s_0_25_0|yalnz hissediyor ...| s_0.trec|  train|[yalnz, hissediyo...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|   11|  0|  s_0_25_0|  0|
| s_0_115_0|akar yakarm zn ya...| s_0.trec|  train|[akar, yakarm, zn...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|   10|  0| s_0_115_0|  0|
|s_10_277_0|       kill yourself|s_10.trec|  train|    [kill, yourself]|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|    9|  0|s_10_277_0|  0|
|s_10_465_3|He eats all the time|s_10.trec|  train|[he, eats, all, t...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|   18|  0|s_10_465_3|  0|
|s_10_473_0|What are

21580

In [59]:
# group by pivot so that we have one row per document with relevance scores
def pivot_training(df, feature_columns):
    scores = (
        df.withColumn("query", F.concat(F.lit("q"), F.col("query").cast("string")))
        .groupBy("docid")
        .pivot("query")
        .agg(F.first("rel"))
    )
    # hash the docid to make it easy to split test train
    return (
        df.groupBy("docid")
        .agg(*[array_to_vector(F.first(c)).alias(c) for c in feature_columns])
        .join(scores, "docid", how="inner")
        .withColumn("sampleid", F.crc32(F.col("docid")) % 100)
    )


prep_df = pivot_training(train_df, ["hashingtf", "tfidf"]).cache()
prep_df.printSchema()
prep_df.show()

root
 |-- docid: string (nullable = true)
 |-- hashingtf: vector (nullable = true)
 |-- tfidf: vector (nullable = true)
 |-- q1: integer (nullable = true)
 |-- q10: integer (nullable = true)
 |-- q11: integer (nullable = true)
 |-- q12: integer (nullable = true)
 |-- q13: integer (nullable = true)
 |-- q14: integer (nullable = true)
 |-- q15: integer (nullable = true)
 |-- q16: integer (nullable = true)
 |-- q17: integer (nullable = true)
 |-- q18: integer (nullable = true)
 |-- q19: integer (nullable = true)
 |-- q2: integer (nullable = true)
 |-- q20: integer (nullable = true)
 |-- q21: integer (nullable = true)
 |-- q3: integer (nullable = true)
 |-- q4: integer (nullable = true)
 |-- q5: integer (nullable = true)
 |-- q6: integer (nullable = true)
 |-- q7: integer (nullable = true)
 |-- q8: integer (nullable = true)
 |-- q9: integer (nullable = true)
 |-- sampleid: long (nullable = true)

+-------------+--------------------+--------------------+----+----+----+----+----+----+----+--

In [60]:
# build a naive bayes model pipeline after removing mean
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

# normalize the data
from pyspark.ml.feature import StandardScaler, MinMaxScaler


def build_nb_model_pipeline(feature, targets):
    pipeline = Pipeline(
        stages=[MinMaxScaler(inputCol=feature, outputCol=f"{feature}_scaled")]
        + [
            NaiveBayes(
                labelCol=target,
                featuresCol=f"{feature}_scaled",
                predictionCol=f"{target}_prediction",
                probabilityCol=f"{target}_probability",
                rawPredictionCol=f"{target}_raw",
                smoothing=1.0,
            )
            for target in targets
        ]
    )
    return pipeline


# 80/20 split
prep_train_df = prep_df.where("sampleid < 80")
prep_test_df = prep_df.where("sampleid >= 80")

pipeline = build_nb_model_pipeline(
    "tfidf", [c for c in prep_df.columns if c.startswith("q")]
)
print(pipeline)
model = pipeline.fit(prep_train_df.fillna(0))
predictions = model.transform(prep_df).cache()
predictions.show(5)

Pipeline_eb7b9275d943


+------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+-------------

In [73]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import tqdm
import numpy as np


def run_evaluation(df, targets):
    results = {
        "f1": {
            "train": {},
            "test": {},
        },
        "accuracy": {"train": {}, "test": {}},
    }
    for c in tqdm.tqdm(targets):
        f1_evaluator = MulticlassClassificationEvaluator(
            labelCol=c, predictionCol=f"{c}_prediction", metricName="f1"
        )
        acc_evaluator = MulticlassClassificationEvaluator(
            labelCol=c, predictionCol=f"{c}_prediction", metricName="accuracy"
        )
        subset = df.where(f"{c} is not null")
        test_subset = subset.where("sampleid >= 80")
        train_subset = subset.where("sampleid < 80")
        results["f1"]["train"][c] = f1_evaluator.evaluate(train_subset)
        results["f1"]["test"][c] = f1_evaluator.evaluate(test_subset)
        results["accuracy"]["train"][c] = acc_evaluator.evaluate(train_subset)
        results["accuracy"]["test"][c] = acc_evaluator.evaluate(test_subset)
    return results


results = run_evaluation(predictions, [c for c in prep_df.columns if c.startswith("q")])
# summarize all the results
(
    results,
    np.mean(list(results["f1"]["test"].values())),
    np.mean(list(results["accuracy"]["test"].values())),
    np.mean(list(results["f1"]["train"].values())),
    np.mean(list(results["accuracy"]["train"].values())),
)

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:23<00:00,  1.14s/it]


({'f1': {'train': {'q1': 0.7592790332115835,
    'q10': 0.6800105020926964,
    'q11': 0.8941352334446162,
    'q12': 0.8911831235315125,
    'q13': 0.9214151523953786,
    'q14': 0.900883164875774,
    'q15': 0.8174762156880939,
    'q16': 0.6915598425686754,
    'q17': 0.8597728725713107,
    'q18': 0.8337573105044516,
    'q19': 0.8850642345787977,
    'q2': 0.8579017264276229,
    'q20': 0.8259058914747465,
    'q21': 0.8598780660809872,
    'q3': 0.7490735746968458,
    'q4': 0.856196757600225,
    'q5': 0.8480137985873437,
    'q6': 0.9645801448541502,
    'q7': 0.7580131745426064,
    'q8': 0.891757264723261,
    'q9': 0.6193349130161816},
   'test': {'q1': 0.7825068332682547,
    'q10': 0.5964566210045663,
    'q11': 0.938453529812197,
    'q12': 0.9493227855687402,
    'q13': 0.9060134477946463,
    'q14': 0.8932390411587967,
    'q15': 0.8511712873415002,
    'q16': 0.676581877712397,
    'q17': 0.8970823341326939,
    'q18': 0.8947101255677264,
    'q19': 0.8598687664041995,

In [77]:
# let's try an alternative of training a new pipeline for each query
def build_fit_nb_model_pipelines(df, feature, targets):
    pipelines = []
    for target in targets:
        pipeline = Pipeline(
            stages=[MinMaxScaler(inputCol=feature, outputCol=f"{feature}_scaled")]
            + [
                NaiveBayes(
                    labelCol=target,
                    featuresCol=f"{feature}_scaled",
                    predictionCol=f"{target}_prediction",
                    probabilityCol=f"{target}_probability",
                    rawPredictionCol=f"{target}_raw",
                    smoothing=1.0,
                )
            ]
        )
        model = pipeline.fit(df.where(f"{target} is not null"))
        pipelines.append(model)
    return pipelines


# 80/20 split
pipelines = build_fit_nb_model_pipelines(
    prep_df.where("sampleid < 80"),
    "tfidf",
    [c for c in prep_df.columns if c.startswith("q")],
)
per_pred = prep_df
for model in pipelines:
    per_pred = model.transform(per_pred).drop("tfidf_scaled")

per_pred = per_pred.cache()
per_pred.show(5)

+------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+-------------

In [78]:
results = run_evaluation(per_pred, [c for c in prep_df.columns if c.startswith("q")])
# summarize all the results
(
    results,
    np.mean(list(results["f1"]["test"].values())),
    np.mean(list(results["accuracy"]["test"].values())),
    np.mean(list(results["f1"]["train"].values())),
    np.mean(list(results["accuracy"]["train"].values())),
)

100%|██████████| 21/21 [00:28<00:00,  1.38s/it]


({'f1': {'train': {'q1': 0.7803252360422663,
    'q10': 0.7759897139214771,
    'q11': 0.895632970691376,
    'q12': 0.9124754520967219,
    'q13': 0.9191618853086329,
    'q14': 0.8997269158559481,
    'q15': 0.8196281020769511,
    'q16': 0.7612694886747505,
    'q17': 0.8585709816708796,
    'q18': 0.8384777313150609,
    'q19': 0.900950444743558,
    'q2': 0.8562740945865234,
    'q20': 0.8281512311978254,
    'q21': 0.8704188604452814,
    'q3': 0.7477495266746426,
    'q4': 0.8549978199258774,
    'q5': 0.8540993241069872,
    'q6': 0.9604012581095789,
    'q7': 0.8036255552942118,
    'q8': 0.8894348240677601,
    'q9': 0.7311617151980775},
   'test': {'q1': 0.7799639216344338,
    'q10': 0.6262726895354542,
    'q11': 0.938453529812197,
    'q12': 0.9444457828422752,
    'q13': 0.9060134477946463,
    'q14': 0.8932390411587967,
    'q15': 0.8600983092961703,
    'q16': 0.6718450766416452,
    'q17': 0.8947649572649573,
    'q18': 0.8947101255677264,
    'q19': 0.859868766404199

In [80]:
# let's try doing a semi-supervised approach
def build_nb_model_pipeline(feature, targets):
    pipeline = Pipeline(
        stages=[MinMaxScaler(inputCol=feature, outputCol=f"{feature}_scaled")]
        + [
            NaiveBayes(
                labelCol=target,
                featuresCol=f"{feature}_scaled",
                predictionCol=f"{target}_prediction",
                probabilityCol=f"{target}_probability",
                rawPredictionCol=f"{target}_raw",
                smoothing=1.0,
            )
            for target in targets
        ]
    )
    return pipeline


# 80/20 split
pipeline = build_nb_model_pipeline(
    "tfidf", [c for c in prep_df.columns if c.startswith("q")]
)
model = pipeline.fit(prep_df.where("sampleid < 80").fillna(0))
predictions = model.transform(prep_df)
# now fill the missing values with the predictions

prep_df_filled = prep_df.join(
    predictions.select(
        "docid", *[f"{c}_prediction" for c in prep_df.columns if c.startswith("q")]
    ),
    "docid",
).select(
    "docid",
    "tfidf",
    *[
        F.coalesce(F.col(c), F.col(f"{c}_prediction")).alias(c)
        for c in prep_df.columns
        if c.startswith("q")
    ],
)

model_filled = pipeline.fit(prep_df_filled.where("sampleid < 80"))
predictions_filled = model_filled.transform(prep_df_filled).cache()
predictions_filled.show(5)

24/03/24 19:53:28 WARN DAGScheduler: Broadcasting large task binary with size 1003.3 KiB
24/03/24 19:53:35 WARN DAGScheduler: Broadcasting large task binary with size 1018.1 KiB


+------------+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------

In [83]:
results = run_evaluation(
    predictions_filled, [c for c in prep_df.columns if c.startswith("q")]
)
# summarize all the results
(
    results,
    np.mean(list(results["f1"]["test"].values())),
    np.mean(list(results["accuracy"]["test"].values())),
    np.mean(list(results["f1"]["train"].values())),
    np.mean(list(results["accuracy"]["train"].values())),
)

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:46<00:00,  2.23s/it]


({'f1': {'train': {'q1': 0.982620379690667,
    'q10': 0.980472131893606,
    'q11': 0.9930917687513431,
    'q12': 0.9928362742808627,
    'q13': 0.9947784258563721,
    'q14': 0.9934363935895847,
    'q15': 0.9875955723814962,
    'q16': 0.9817716328910319,
    'q17': 0.9912754104263787,
    'q18': 0.9899660685198705,
    'q19': 0.9928895373617144,
    'q2': 0.9899313002648917,
    'q20': 0.9888162937887194,
    'q21': 0.9916762364963458,
    'q3': 0.9851064358162884,
    'q4': 0.9908503157549086,
    'q5': 0.9920698567317897,
    'q6': 0.9976934152405608,
    'q7': 0.9845791230430141,
    'q8': 0.9929414506841515,
    'q9': 0.9770070930851728},
   'test': {'q1': 0.9864934768781418,
    'q10': 0.9710611448955424,
    'q11': 0.9958196210858813,
    'q12': 0.9971068307553024,
    'q13': 0.9934431130474657,
    'q14': 0.9936900120689914,
    'q15': 0.9911223816853083,
    'q16': 0.9803876602457239,
    'q17': 0.9936199999182052,
    'q18': 0.9939055737790612,
    'q19': 0.99158738572365

In [84]:
# let's try our model with just the hashingtf features
pipeline_hashing = build_nb_model_pipeline(
    "hashingtf", [c for c in prep_df.columns if c.startswith("q")]
)
model_hashing = pipeline_hashing.fit(prep_df.where("sampleid < 80").fillna(0))
predictions_hashing = model_hashing.transform(prep_df)

results = run_evaluation(
    predictions_hashing, [c for c in prep_df.columns if c.startswith("q")]
)
# summarize all the results
(
    results,
    np.mean(list(results["f1"]["test"].values())),
    np.mean(list(results["accuracy"]["test"].values())),
    np.mean(list(results["f1"]["train"].values())),
    np.mean(list(results["accuracy"]["train"].values())),
)

100%|██████████| 21/21 [00:17<00:00,  1.20it/s]


({'f1': {'train': {'q1': 0.7592790332115835,
    'q10': 0.6800105020926964,
    'q11': 0.8941352334446162,
    'q12': 0.8911831235315125,
    'q13': 0.9214151523953786,
    'q14': 0.900883164875774,
    'q15': 0.8174762156880939,
    'q16': 0.6915598425686754,
    'q17': 0.8597728725713107,
    'q18': 0.8337573105044516,
    'q19': 0.8850642345787977,
    'q2': 0.8579017264276229,
    'q20': 0.8259058914747465,
    'q21': 0.8598780660809872,
    'q3': 0.7490735746968458,
    'q4': 0.856196757600225,
    'q5': 0.8480137985873437,
    'q6': 0.9645801448541502,
    'q7': 0.7580131745426064,
    'q8': 0.891757264723261,
    'q9': 0.6193349130161816},
   'test': {'q1': 0.7825068332682547,
    'q10': 0.5964566210045663,
    'q11': 0.938453529812197,
    'q12': 0.9493227855687402,
    'q13': 0.9060134477946463,
    'q14': 0.8932390411587967,
    'q15': 0.8511712873415002,
    'q16': 0.676581877712397,
    'q17': 0.8970823341326939,
    'q18': 0.8947101255677264,
    'q19': 0.8598687664041995,